In [1]:
import tensorflow_datasets as tfds

In [2]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
tokenizer = info.features['text'].encoder

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVN11D5/imdb_reviews-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVN11D5/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVN11D5/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [7]:
BUFFER_SIZE = 10000
BATCH_SIZE = 256

train_data, test_data = dataset['train'], dataset['test']

In [8]:
train_dataset = train_data.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_data.padded_batch(BATCH_SIZE)

In [9]:
import tensorflow as tf
import numpy as np

In [11]:
batch_size = 1
timesteps = 20
features = 16
lstm_dim = 8

print(f'Batch size: {batch_size}')
print(f'Timesteps (sequence length): {timesteps}')
print(f'Features (embedding size): {features}')
print(f'LSTM output units: {lstm_dim}')

random_input = np.random.rand(batch_size, timesteps, features)
print(f'Shape of the array: {random_input.shape}')

lstm = tf.keras.layers.LSTM(lstm_dim)
result = lstm(random_input)
print(f'Shape of the LSTM output with return_sequences=False : {result.shape}')

lstm_rs = tf.keras.layers.LSTM(lstm_dim, return_sequences=True)
result = lstm_rs(random_input)
print(f'Shape of the LSTM output with return_sequences=True : {result.shape}')

Batch size: 1
Timesteps (sequence length): 20
Features (embedding size): 16
LSTM output units: 8
Shape of the array: (1, 20, 16)
Shape of the LSTM output with return_sequences=False : (1, 8)
Shape of the LSTM output with return_sequences=True : (1, 20, 8)


In [12]:
import tensorflow as tf

embedding_dim = 64
lstm1_dim = 64
lstm2_dim = 32
dense_dim = 64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm1_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm2_dim)),
    tf.keras.layers.Dense(dense_dim, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          523840    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 635,329
Trainable params: 635,329
Non-trai

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data = test_dataset)

Epoch 1/10
98/98 [==============================] - 66s 568ms/step - loss: 0.6260 - accuracy: 0.6367 - val_loss: 0.7030 - val_accuracy: 0.4996
Epoch 2/10
98/98 [==============================] - 47s 476ms/step - loss: 0.6125 - accuracy: 0.6522 - val_loss: 0.6193 - val_accuracy: 0.6391
Epoch 3/10
98/98 [==============================] - 44s 447ms/step - loss: 0.5889 - accuracy: 0.6923 - val_loss: 0.6811 - val_accuracy: 0.5000
Epoch 4/10
98/98 [==============================] - 43s 435ms/step - loss: 0.5747 - accuracy: 0.6880 - val_loss: 0.6004 - val_accuracy: 0.6750
Epoch 5/10
98/98 [==============================] - 42s 423ms/step - loss: 0.4712 - accuracy: 0.7860 - val_loss: 0.4995 - val_accuracy: 0.7783
Epoch 6/10
98/98 [==============================] - 51s 524ms/step - loss: 0.3977 - accuracy: 0.8381 - val_loss: 0.4767 - val_accuracy: 0.7882
Epoch 7/10
98/98 [==============================] - 41s 416ms/step - loss: 0.3397 - accuracy: 0.8623 - val_loss: 0.4863 - val_accuracy: 0.7919